In [1]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from torch.utils.data import Dataset
from torch import nn
from torch.optim import Adam
from datasets import load_dataset
import random
from copy import deepcopy
import optuna
from torch.utils.data import DataLoader
import wandb

In [2]:
# Initialize W&B
wandb.init(
    project="few-shot-multiset",
    name="maml-multiset-optuna",
)

wandb: Currently logged in as: kostic-stojan23 (kostic-stojan23-university-of-belgrade). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
def load_and_normalize(dataset_name):
    dataset = load_dataset(dataset_name)

    if dataset_name == "amazon_polarity":
        def process(example):
            return {"text": example["content"], "label": example["label"]}
    elif dataset_name == "yelp_polarity":
        def process(example):
            return {"text": example["text"], "label": example["label"]}
    elif dataset_name == "sentiment140":
        def process(example):
            label = 0 if example["sentiment"] == 0 else 1  # Convert (0,4) to (0,1)
            return {"text": example["text"], "label": label}

    dataset["train"] = dataset["train"].map(process)
    dataset["test"] = dataset["test"].map(process)

    return dataset["train"], dataset["test"]

In [4]:
dataset_names = ["amazon_polarity", "yelp_polarity", "sentiment140"]
train_data, test_data = {}, {}

for name in dataset_names:
    train_data[name], test_data[name] = load_and_normalize(name)

In [5]:
class FewShotDataset(Dataset):
    def __init__(self, data, num_support, num_query):
        self.data = data
        self.num_support = num_support
        self.num_query = num_query

    def get_task(self):
        indices = list(range(len(self.data)))
        random.shuffle(indices)

        support_indices = indices[:self.num_support]
        query_indices = indices[self.num_support:self.num_support + self.num_query]

        support_set = [(self.data[i]['text'], self.data[i]['label']) for i in support_indices]
        query_set = [(self.data[i]['text'], self.data[i]['label']) for i in query_indices]

        return support_set, query_set

In [6]:
model_path = "NLP_VER1"
base_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model.to(device)

def inner_loop(model, support_set, num_steps=1, lr=1e-5):
    task_model = deepcopy(model).to(device)  # Move task-specific model to GPU
    optimizer = torch.optim.Adam(task_model.parameters(), lr=lr)
    loss_fn = torch.nn.CrossEntropyLoss()

    for _ in range(num_steps):
        for text, label in support_set:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
            labels = torch.tensor([label]).unsqueeze(0).to(device)  # Move labels to GPU

            outputs = task_model(**inputs, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return task_model

In [8]:
def outer_loop(meta_model, tasks, meta_optimizer, num_inner_steps=1):
    meta_optimizer.zero_grad()
    total_loss = 0

    for support_set, query_set in tasks:
        task_model = inner_loop(meta_model, support_set, num_steps=num_inner_steps)

        for text, label in query_set:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
            labels = torch.tensor([label]).unsqueeze(0).to(device)

            outputs = task_model(**inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss

    total_loss.backward()
    meta_optimizer.step()

    return total_loss.item()

In [9]:
def objective(trial):
    meta_lr = trial.suggest_loguniform("meta_lr", 1e-5, 1e-3)
    inner_lr = trial.suggest_loguniform("inner_lr", 1e-5, 1e-3)
    num_support = trial.suggest_int("num_support", 1, 10)
    num_query = trial.suggest_int("num_query", 1, 10)
    inner_steps = trial.suggest_int("inner_steps", 1, 5)
    batch_size = trial.suggest_int("batch_size", 2, 8)
    meta_epochs = trial.suggest_int("meta_epochs", 5, 10)

    dataset_name = random.choice(dataset_names)
    few_shot_dataset = FewShotDataset(train_data[dataset_name], num_support, num_query)

    meta_optimizer = Adam(base_model.parameters(), lr=meta_lr)

    for epoch in range(meta_epochs):
        tasks = [few_shot_dataset.get_task() for _ in range(batch_size)]
        loss = outer_loop(base_model, tasks, meta_optimizer, num_inner_steps=inner_steps)

        wandb.log({
            "trial": trial.number,
            "epoch": epoch + 1,
            "meta_loss": loss,
            "meta_lr": meta_lr,
            "inner_lr": inner_lr,
            "num_support": num_support,
            "num_query": num_query,
            "inner_steps": inner_steps,
            "batch_size": batch_size,
        })

        print(f"[Trial {trial.number}, Epoch {epoch + 1}] Loss: {loss:.4f}")

    return loss

In [11]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=34)

best_trial = study.best_trial
print(f"Best Trial: {best_trial.number}, Loss: {best_trial.value}")
print("Best Hyperparameters:", best_trial.params)

[I 2025-01-30 21:52:40,410] A new study created in memory with name: no-name-25124f3e-a74c-4f17-82ff-59787b330d7e
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_16660\974922913.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  meta_lr = trial.suggest_loguniform("meta_lr", 1e-5, 1e-3)
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_16660\974922913.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  inner_lr = trial.suggest_loguniform("inner_lr", 1e-5, 1e-3)


[Trial 0, Epoch 1] Loss: 23.6549
[Trial 0, Epoch 2] Loss: 4.3525
[Trial 0, Epoch 3] Loss: 15.8666
[Trial 0, Epoch 4] Loss: 11.7129
[Trial 0, Epoch 5] Loss: 6.9439
[Trial 0, Epoch 6] Loss: 9.8187
[Trial 0, Epoch 7] Loss: 5.4228
[Trial 0, Epoch 8] Loss: 12.7705
[Trial 0, Epoch 9] Loss: 4.7034


[I 2025-01-30 22:00:25,321] Trial 0 finished with value: 4.703431606292725 and parameters: {'meta_lr': 0.00012765534849510702, 'inner_lr': 1.9468831996027415e-05, 'num_support': 9, 'num_query': 4, 'inner_steps': 5, 'batch_size': 6, 'meta_epochs': 9}. Best is trial 0 with value: 4.703431606292725.


[Trial 1, Epoch 1] Loss: 34.0209
[Trial 1, Epoch 2] Loss: 13.9256
[Trial 1, Epoch 3] Loss: 13.2510
[Trial 1, Epoch 4] Loss: 8.4297
[Trial 1, Epoch 5] Loss: 12.9661
[Trial 1, Epoch 6] Loss: 18.4708


[I 2025-01-30 22:02:28,265] Trial 1 finished with value: 18.4708251953125 and parameters: {'meta_lr': 1.9320689557340608e-05, 'inner_lr': 0.00019001272780410218, 'num_support': 6, 'num_query': 8, 'inner_steps': 4, 'batch_size': 5, 'meta_epochs': 6}. Best is trial 0 with value: 4.703431606292725.


...

[Trial 32, Epoch 1] Loss: 1.1204
[Trial 32, Epoch 2] Loss: 0.9243
[Trial 32, Epoch 3] Loss: 8.6580
[Trial 32, Epoch 4] Loss: 5.2228
[Trial 32, Epoch 5] Loss: 2.1078
[Trial 32, Epoch 6] Loss: 8.3319
[Trial 32, Epoch 7] Loss: 6.8379
[Trial 32, Epoch 8] Loss: 5.0200
[Trial 32, Epoch 9] Loss: 6.1424
[Trial 32, Epoch 10] Loss: 6.6841


[I 2025-01-31 00:43:40,338] Trial 32 finished with value: 6.684129238128662 and parameters: {'meta_lr': 0.0006922877388649675, 'inner_lr': 4.230670857459767e-05, 'num_support': 2, 'num_query': 1, 'inner_steps': 1, 'batch_size': 8, 'meta_epochs': 10}. Best is trial 21 with value: 0.0828155130147934.


[Trial 33, Epoch 1] Loss: 15.6074
[Trial 33, Epoch 2] Loss: 21.4301
[Trial 33, Epoch 3] Loss: 12.6821
[Trial 33, Epoch 4] Loss: 15.6322
[Trial 33, Epoch 5] Loss: 6.1121
[Trial 33, Epoch 6] Loss: 13.2195
[Trial 33, Epoch 7] Loss: 15.3118
[Trial 33, Epoch 8] Loss: 13.5080
[Trial 33, Epoch 9] Loss: 7.3450
[Trial 33, Epoch 10] Loss: 17.0412


[I 2025-01-31 00:50:56,025] Trial 33 finished with value: 17.041229248046875 and parameters: {'meta_lr': 0.0004067884856220484, 'inner_lr': 2.4374013218275132e-05, 'num_support': 1, 'num_query': 2, 'inner_steps': 1, 'batch_size': 8, 'meta_epochs': 10}. Best is trial 21 with value: 0.0828155130147934.


Best Trial: 21, Loss: 0.0828155130147934
Best Hyperparameters: {'meta_lr': 0.0004379174327349713, 'inner_lr': 4.4169684443381925e-05, 'num_support': 1, 'num_query': 1, 'inner_steps': 1, 'batch_size': 8, 'meta_epochs': 10}


In [12]:
best_params = best_trial.params
dataset_name = random.choice(dataset_names)
few_shot_dataset = FewShotDataset(train_data[dataset_name], best_params["num_support"], best_params["num_query"])
meta_optimizer = Adam(base_model.parameters(), lr=best_params["meta_lr"])

for epoch in range(best_params["meta_epochs"]):
    tasks = [few_shot_dataset.get_task() for _ in range(best_params["batch_size"])]
    loss = outer_loop(base_model, tasks, meta_optimizer, num_inner_steps=best_params["inner_steps"])
    wandb.log({"epoch": epoch + 1, "meta_loss": loss})
    print(f"Epoch {epoch + 1}, Meta Loss: {loss:.4f}")

Epoch 1, Meta Loss: 4.1011
Epoch 2, Meta Loss: 3.2963
Epoch 3, Meta Loss: 4.5106
Epoch 4, Meta Loss: 3.4007
Epoch 5, Meta Loss: 7.4569
Epoch 6, Meta Loss: 9.8577
Epoch 7, Meta Loss: 5.2001
Epoch 8, Meta Loss: 11.9975
Epoch 9, Meta Loss: 10.9002
Epoch 10, Meta Loss: 6.7005


In [13]:
new_dataset_name = random.choice(dataset_names)
new_task_data = FewShotDataset(test_data[new_dataset_name], best_params["num_support"], best_params["num_query"])
new_support_set, new_query_set = new_task_data.get_task()

adapted_model = inner_loop(base_model, new_support_set, num_steps=best_params["inner_steps"])
adapted_model = adapted_model.to(device)
total_loss = 0

for text, label in new_query_set:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    labels = torch.tensor([label]).unsqueeze(0).to(device)


    outputs = adapted_model(**inputs, labels=labels)
    loss = outputs.loss
    total_loss += loss

wandb.log({"new_task_loss": total_loss.item()})
print(f"New Task ({new_dataset_name}) Evaluation Loss: {total_loss.item():.4f}")

New Task (amazon_polarity) Evaluation Loss: 0.0311


In [ ]:
#This code works but it takes hours to complete because of milions of instances
from sklearn.metrics import accuracy_score, classification_report
def evaluate_model_on_all_datasets(model, test_data):
    model.eval()
    all_preds = []
    all_labels = []

    for dataset_name in ["sentiment140", "amazon_polarity", "yelp_polarity"]:
        encodings = tokenizer(test_data[dataset_name]["text"], padding=True, truncation=True, return_tensors="pt", max_length=512)

        test_dataset = torch.utils.data.TensorDataset(encodings.input_ids, encodings.attention_mask, torch.tensor(test_data[dataset_name]["label"]))
        test_loader = DataLoader(test_dataset, batch_size=16)

        with torch.no_grad():
            for batch in test_loader:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(model.device)
                attention_mask = attention_mask.to(model.device)
                labels = labels.to(model.device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        wandb.log({f"{dataset_name}_accuracy": accuracy})

        print(f"Evaluation on {dataset_name}: Accuracy = {accuracy:.4f}")
        print(f"Classification Report for {dataset_name}:")
        print(classification_report(all_labels, all_preds))

evaluate_model_on_all_datasets(adapted_model, test_data)

In [38]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# 20 unseen sentences for testing
test_sentences = [
    "The customer service was beyond my expectations!",
    "I waited an hour for my order, and it was still wrong.",
    "This new phone update is a complete disaster.",
    "I'm absolutely thrilled with my new laptop!",
    "The food was bland and overpriced, not coming back.",
    "Best vacation spot ever, can't wait to return!",
    "The product broke after just two uses, very disappointed.",
    "Excellent book, well-written and engaging.",
    "Movie was predictable and boring, nothing special.",
    "Customer support was extremely helpful and quick to respond.",
    "I regret purchasing this item, waste of money.",
    "One of the best restaurants in town, highly recommended!",
    "The new policy changes are frustrating and unnecessary.",
    "I love how comfortable and stylish these shoes are!",
    "The concert was an unforgettable experience.",
    "The software crashes frequently, making it unusable.",
    "Brilliant storytelling, kept me hooked from start to finish.",
    "Shipping took forever, and the package arrived damaged.",
    "Great workout program, helped me get in shape quickly.",
    "The app's interface is confusing and hard to navigate."
]

inputs = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inputs = {key: value.to(device) for key, value in inputs.items()}

with torch.no_grad():
    outputs = adapted_model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

if isinstance(predictions, torch.Tensor):
    predictions = predictions.cpu().numpy()
predictions = predictions.tolist()

import torch.nn.functional as F

probs = F.softmax(logits, dim=1).cpu().numpy()
for i, (sentence, prob) in enumerate(zip(test_sentences, probs)):
    print(f"{i+1}. {sentence}\n   ➝ Positive: {prob[1]:.2f}, Negative: {prob[0]:.2f}\n")


1. The customer service was beyond my expectations!
   ➝ Positive: 0.92, Negative: 0.08

2. I waited an hour for my order, and it was still wrong.
   ➝ Positive: 0.01, Negative: 0.99

3. This new phone update is a complete disaster.
   ➝ Positive: 0.00, Negative: 1.00

4. I'm absolutely thrilled with my new laptop!
   ➝ Positive: 0.98, Negative: 0.02

5. The food was bland and overpriced, not coming back.
   ➝ Positive: 0.00, Negative: 1.00

6. Best vacation spot ever, can't wait to return!
   ➝ Positive: 0.99, Negative: 0.01

7. The product broke after just two uses, very disappointed.
   ➝ Positive: 0.00, Negative: 1.00

8. Excellent book, well-written and engaging.
   ➝ Positive: 1.00, Negative: 0.00

9. Movie was predictable and boring, nothing special.
   ➝ Positive: 0.00, Negative: 1.00

10. Customer support was extremely helpful and quick to respond.
   ➝ Positive: 0.98, Negative: 0.02

11. I regret purchasing this item, waste of money.
   ➝ Positive: 0.00, Negative: 1.00

12. O

In [17]:
wandb.finish()

batch_size,▆▅▁▃▁▃▁▅▇██████▇▇▇▆▆▇▇▇████▇██▆▇██▂█████
epoch,▃▃▃▁▃▄▄▃▃▁▁▂▄▇▅▃▄▁▆▂▂▃▄▅▃▇█▁▃▇▄▄▂▁▂▄▁█▁▇
inner_lr,▁▂▂▁▂▄█▅▅▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▃▁▁▁▁▁▁▁
inner_steps,█▆▁▁▁▃██▁▁▃▁▁▁▃▁▃▃▆▁▁▁▁▁▃▁▁▃▃▃▅▅▅▅▁▁▁▁▁▁
meta_loss,▂▂▁▂▂▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▇▆█▁▂▂▁▂▂▁▂▁▁▁▂▂▁▁
meta_lr,▂▂▁▂▂▂▂▂▂▂▃▃▃▃▃▅▃▃▁▁▃▃▄▄▄▅▆▄▄▂███▂▅▅▅▆▆▄
new_task_loss,▁
num_query,▃▃▆▂▂▆▅▅▆▆▇▇▁▂▂▁▃▃▄▄▂█▃▁▁▂▂▁▁▁▄▄▂▃▃▂▁▁▂▂
num_support,█▅▅▃▅▅▄▅▅▃▁▁▁▁▁▂▂▇▂▂▂▁▃▂▂▁▂▂▂▂▃▄▁▁▁▄▁▁▂▁
sentiment140_accuracy,▁
trial,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
